<a href="https://colab.research.google.com/github/ghr8635/E2E-DriveAI-ROS2-based-Modular-Framework-for-Autonomous-Vehicle-Control/blob/main/inference_from_resent_feature_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

In [6]:
my_model = r'C:\Users\hussa\OneDrive\Desktop\Projects\ROS2-Modular-Framework-for-End-to-End-Autonomous-Vehicle-Control-from-Raw-Sensor-Data\resnet18\trained models\feature_extractor_resnet18.pth'

model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 10)
model.load_state_dict(torch.load(my_model), strict=True)
model.fc = torch.nn.Identity()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

c:\Users\hussa\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\hussa\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [11]:
my_image = r"C:\Users\hussa\OneDrive\Desktop\Projects\Project\MOT\INFRA-3DRC_scene-15\INFRA-3DRC_scene-15\camera_01\camera_01__data\camera_01__2023-06-02-21-28-11-114.png"

transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),  # Adjust to your model's input size
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

img = Image.open(my_image)

# Apply transformations
img = transform(img)

# Add batch dimension
img = img.unsqueeze(0)
img = img.to(device)




In [14]:
features = model(img)
features = features [0]


print(features.shape)
print(features)

torch.Size([512])
tensor([0.8432, 0.7705, 0.8087, 1.0307, 0.7593, 0.8299, 0.7650, 1.0814, 0.7951,
        0.7536, 0.6781, 0.7696, 0.7543, 0.9128, 0.8330, 0.8541, 0.7809, 1.0725,
        0.7268, 0.5710, 0.7486, 0.8657, 0.8775, 0.8136, 0.7527, 0.8423, 0.8345,
        0.8055, 0.7666, 0.6872, 0.6337, 0.8444, 0.8120, 0.7712, 0.7215, 0.6396,
        0.6078, 0.6671, 0.6608, 0.7998, 0.6520, 0.7142, 0.7945, 0.7916, 0.7462,
        0.7524, 0.7729, 0.9833, 0.9092, 0.7399, 0.9664, 0.8597, 0.8047, 0.7353,
        0.7613, 0.8924, 1.0821, 0.7310, 0.8786, 0.6628, 0.6973, 0.8702, 0.7945,
        0.8143, 0.8759, 0.9096, 0.5860, 0.8388, 0.8558, 0.7402, 0.6586, 0.5262,
        0.7420, 0.7896, 0.7153, 0.7479, 0.7792, 0.7705, 0.7381, 0.7831, 0.7165,
        0.8697, 0.9609, 1.0684, 0.6012, 0.7590, 0.8057, 0.7616, 0.7648, 0.6581,
        0.6460, 0.7218, 0.7948, 0.6638, 0.6547, 0.7741, 0.6963, 0.6778, 0.8209,
        0.6950, 0.9124, 0.9453, 0.9559, 0.7661, 0.6942, 0.6845, 0.7830, 0.8699,
        0.7512, 0.7365